In [1]:
from fastai.vision.all import *
from pathlib import Path
import gzip
import os
import torch
import numpy as np

## Download and load data

In [2]:
dir_name = Path("/home/jedrzej/Desktop/my_projects/FashionMnist_nn")
train_imgs_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz"
train_labels_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz"
test_imgs_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz"
test_labels_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz"

In [3]:
# Load mnist data from path
def load_mnist(path, kind='train') -> (torch.Tensor, torch.Tensor):

    labels_path = os.path.join(path,
                               '%s_labels.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s_images.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 28, 28)

    return torch.from_numpy(images), torch.from_numpy(labels)

In [4]:
for (url, name) in [(train_imgs_url, "train_images.gz"),
                    (train_labels_url, "train_labels.gz"),
                    (test_imgs_url, "test_images.gz"),
                    (test_labels_url, "test_labels.gz")]:
    
    download_url(url, dir_name/name)

In [5]:
train_images, train_labels = load_mnist(dir_name)
test_images, test_labels = load_mnist(dir_name, kind="test")

/home/jedrzej/.local/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)


In [6]:
images = torch.cat((train_images, test_images), dim=0)
labels = torch.cat((train_labels, test_labels), dim=0)
data = list(zip(list(images), list(labels)))

## Inspect batch

In [7]:
def to_image(x):
    "Convert a tensor or array to a PIL int8 Image"
    if isinstance(x,Image.Image): return x
    if isinstance(x,Tensor): x = to_np(x)
    if x.dtype==np.float32: x = (x*255).astype(np.uint8)
    return Image.fromarray(x, mode=['L'][x.shape[0]==4])

In [12]:
def split_train_test(a):
    print(a)
    return list(range(60000)), list(range(10000))

In [9]:
first_image = images[0]
to_image(first_image)

In [10]:
# ds = Datasets(data)
# dl = ds.dataloaders(batch_size=64, shuffle_train=True)


In [ ]:
db = DataBlock(blocks = (ImageBlock, CategoryBlock),
               splitter = split_train_test,
               )
db.datasets(data)

In [ ]:
db.datasets.train